In [3]:
import pandas as pd 
import numpy as np
from dotenv import load_dotenv
import google.generativeai as genai
import os

In [4]:
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-1.5-flash")
# response = model.generate_content("Explain how AI works")
# print(response.text)

In [5]:
df=pd.DataFrame(pd.read_csv("fake_vector_db.csv"))


In [6]:
# df["Embedding"]=df["Embedding"].apply(lambda x: np.array(x))

In [7]:
def get_embedding(text):
    embedded_content = genai.embed_content(
            model="models/text-embedding-004",
            content={text})
    return embedded_content['embedding']


In [8]:
question="How many respondents feel that online food guide saves money?"
embedded_question= get_embedding(question)

In [9]:
# fix_this=df["Embedding"][0]
# try_me = ast.literal_eval(fix_this)
import ast


In [10]:
# for i in range(len(df["Embedding"])):
#     df["Embedding"][i]=ast.literal_eval(df["Embedding"][i])
df["Embedding"]=df["Embedding"].apply(lambda x: ast.literal_eval(x))

In [11]:
df["UseMe"]=df["Embedding"].apply(lambda x:x[0])


In [12]:
df["Closeness"]=df["UseMe"].apply(lambda x: np.dot(x,embedded_question[0]))

In [13]:
df["Closeness"].head()


0    0.461170
1    0.549081
2    0.459530
3    0.489319
4    0.433289
Name: Closeness, dtype: float64

In [14]:
df.sort_values(by="Closeness", ascending=False, inplace=True)
context=""
for i in range (0,5):
    context+=df["Content"][i]


In [15]:
# Now let's generate the response
answer = model.generate_content(f"You are a customer support assistance from the organisation of Zomato, a food delivery service. Use first person like 'We' to answer queries. Don't answer beyond the given context. The context is :{context}. Customer Question: {question}")
# print(answer)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "I'm sorry, but the provided text is a research paper abstract about Zomato and the use of machine learning algorithms to analyze restaurant data.  It doesn't contain information about how many respondents feel online food guides save money.  We don't have access to that specific data.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.08117339611053467
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 415,
        "candidates_token_count": 60,
        "total_token_count": 475
      },
      "model_version": "gemini-1.5-flash"
    }),
)


In [16]:
print(answer.candidates[0].content.parts[0].text)

I'm sorry, but the provided text is a research paper abstract about Zomato and the use of machine learning algorithms to analyze restaurant data.  It doesn't contain information about how many respondents feel online food guides save money.  We don't have access to that specific data.

